In [1]:
import os
import torch
import gym
import numpy as np
from TD3_torch.TD3 import TD3
from PIL import Image
from TD3_torch.utils import ReplayBuffer

env_name = 'BipedalWalkerHardcore-v2'
log_interval = 100           # print avg reward after interval
random_seed = 0
gamma = 0.99                # discount for future rewards
batch_size = 128        # num of transitions sampled from replay buffer
exploration_noise = 0.1 
polyak_int = [0.995, 0.999999]              # target policy update parameter (1-tau)
policy_noise = 0.2          # target policy smoothing noise
noise_clip = 0.5
policy_delay = 2            # delayed policy updates parameter
max_episodes = 50000         # max num of episodes
max_timesteps = 2000        # max timesteps in one episode
max_buffer_length = 300000
directory = "./preTrained/td3_torch/{}".format(env_name) # save trained models
filename = "TD3_torch_{}_{}".format(env_name, random_seed)
reward_history = []



def train():
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    polyak = polyak_int[0]
    
    policy = TD3(state_dim, action_dim, max_action)
    replay_buffer = ReplayBuffer(max_length=max_buffer_length)
    
    print("action_space={}".format(env.action_space))
    print("obs_space={}".format(env.observation_space))
    print("threshold={} \n".format(env.spec.reward_threshold))
    
    if random_seed:
        print("Random Seed: {}".format(random_seed))
        env.seed(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
    
    # logging variables:        
    log_f = open("log.txt","w+")
    
    # training procedure:
    for episode in range(1, max_episodes+1):
        ep_reward = 0
        state = env.reset()
       
        for t in range(max_timesteps):
            # select action and add exploration noise:
            action = policy.select_action(state)
            action = action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0])
            action = action.clip(env.action_space.low, env.action_space.high)
            
            # take action in env:
            next_state, reward, done, _ = env.step(action)
            replay_buffer.add((state, action, reward, next_state, float(done)))
            state = next_state
            
            ep_reward += reward
            
            # if episode is done then update policy:
            if done or t==(max_timesteps-1):
                policy.update(replay_buffer, t, batch_size, gamma, polyak, policy_noise, noise_clip, policy_delay)
                break
        
        reward_history.append(ep_reward)
        avg_reward = np.mean(reward_history[-100:]) 
        
        # logging updates:        
        log_f.write('{},{}\n'.format(episode, ep_reward))
        log_f.flush()
       
        
        # if avg reward > 300 then save and stop traning:
        if avg_reward >= env.spec.reward_threshold: 
            print("########## Solved! ###########")
            name = filename + '_solved'
            policy.save(directory, name)
            log_f.close()
            break
            
        # Calculate polyak
        #part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
        #if part > 1:
        #    part = 1
        #polyak = polyak_int[0] + (1 - part) * (polyak_int[1] - polyak_int[0])     
        
        
        if episode > 500:
            policy.save(directory, filename)
        
        # print avg reward every log interval:
        if episode % log_interval == 0:            
            print("Ep: {}   Rew: {:3.2f}   Avg Rew: {:3.2f}   Polyak: {:6.6f}   Buffer: {:3.2f}   Loss: {:6.6f}  {:6.6f}  {:6.6f}".format(
                episode, ep_reward, avg_reward, polyak, replay_buffer.get_fill(), policy.actor_loss, policy.loss_Q1, policy.loss_Q2))

train()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space=Box(4,)
obs_space=Box(24,)
threshold=300 

Ep: 1   Rew: -93.89   Avg Rew: -93.89   Polyak: 0.995000   Buffer: 0.03   Loss: -0.049667  0.653095  0.499643
Ep: 2   Rew: -102.07   Avg Rew: -97.98   Polyak: 0.995000   Buffer: 0.05   Loss: 0.811679  5.742258  4.799824
Ep: 3   Rew: -106.70   Avg Rew: -100.89   Polyak: 0.995000   Buffer: 0.07   Loss: 0.152485  129.301544  131.299652
Ep: 4   Rew: -216.20   Avg Rew: -129.71   Polyak: 0.995000   Buffer: 0.73   Loss: 1.689378  1.940202  1.534711
Ep: 5   Rew: -180.06   Avg Rew: -139.78   Polyak: 0.995000   Buffer: 1.40   Loss: -0.110850  2.879950  4.398229
Ep: 6   Rew: -109.45   Avg Rew: -134.73   Polyak: 0.995000   Buffer: 1.43   Loss: -0.182587  1.206853  0.881050
Ep: 7   Rew: -108.71   Avg Rew: -131.01   Polyak: 0.995000   Buffer: 1.45   

Ep: 72   Rew: -99.90   Avg Rew: -108.01   Polyak: 0.995000   Buffer: 3.91   Loss: 10.444903  5.820513  6.067129
Ep: 73   Rew: -112.78   Avg Rew: -108.08   Polyak: 0.995000   Buffer: 3.93   Loss: 8.105766  6.760255  6.717532
Ep: 74   Rew: -119.13   Avg Rew: -108.23   Polyak: 0.995000   Buffer: 3.95   Loss: 10.438009  7.168272  3.984973
Ep: 75   Rew: -98.47   Avg Rew: -108.10   Polyak: 0.995000   Buffer: 3.98   Loss: 9.493410  46.886211  51.052864
Ep: 76   Rew: -113.08   Avg Rew: -108.16   Polyak: 0.995000   Buffer: 4.00   Loss: 6.605355  3.912993  4.482635
Ep: 77   Rew: -80.28   Avg Rew: -107.80   Polyak: 0.995000   Buffer: 4.67   Loss: 10.597369  4.402636  5.667572
Ep: 78   Rew: -113.65   Avg Rew: -107.87   Polyak: 0.995000   Buffer: 4.69   Loss: 12.132183  17.143040  16.486115
Ep: 79   Rew: -101.35   Avg Rew: -107.79   Polyak: 0.995000   Buffer: 4.72   Loss: 13.081518  8.796448  12.347109
Ep: 80   Rew: -101.43   Avg Rew: -107.71   Polyak: 0.995000   Buffer: 4.75   Loss: 10.712696  5.2

Ep: 144   Rew: -135.72   Avg Rew: -118.41   Polyak: 0.995000   Buffer: 36.69   Loss: 105.292252  16.050594  18.231659
Ep: 145   Rew: -133.98   Avg Rew: -118.76   Polyak: 0.995000   Buffer: 36.74   Loss: 104.647514  20.565275  16.852905
Ep: 146   Rew: -137.15   Avg Rew: -119.11   Polyak: 0.995000   Buffer: 36.82   Loss: 104.643211  24.415766  22.192429
Ep: 147   Rew: -104.27   Avg Rew: -119.15   Polyak: 0.995000   Buffer: 36.85   Loss: 105.787819  18.793816  16.952572
Ep: 148   Rew: -131.50   Avg Rew: -119.45   Polyak: 0.995000   Buffer: 36.91   Loss: 104.678719  20.464457  18.539234
Ep: 149   Rew: -127.91   Avg Rew: -119.72   Polyak: 0.995000   Buffer: 36.96   Loss: 104.291763  14.936949  17.209238
Ep: 150   Rew: -123.01   Avg Rew: -119.93   Polyak: 0.995000   Buffer: 37.00   Loss: 105.637001  19.166443  16.024929
Ep: 151   Rew: -140.34   Avg Rew: -120.33   Polyak: 0.995000   Buffer: 37.67   Loss: 108.887810  18.269855  20.093084
Ep: 152   Rew: -107.79   Avg Rew: -120.41   Polyak: 0.99

Ep: 214   Rew: -100.28   Avg Rew: -116.46   Polyak: 0.995000   Buffer: 40.52   Loss: 118.236862  23.475193  23.056852
Ep: 215   Rew: -103.11   Avg Rew: -116.28   Polyak: 0.995000   Buffer: 40.53   Loss: 120.475182  25.644625  23.051168
Ep: 216   Rew: -102.61   Avg Rew: -116.01   Polyak: 0.995000   Buffer: 40.55   Loss: 119.902901  24.066017  25.666777
Ep: 217   Rew: -103.04   Avg Rew: -115.90   Polyak: 0.995000   Buffer: 40.57   Loss: 120.901855  27.903450  28.013779
Ep: 218   Rew: -99.86   Avg Rew: -115.80   Polyak: 0.995000   Buffer: 40.58   Loss: 120.488556  19.186399  19.134037
Ep: 219   Rew: -104.99   Avg Rew: -115.70   Polyak: 0.995000   Buffer: 40.60   Loss: 120.054024  23.694767  25.266155
Ep: 220   Rew: -100.18   Avg Rew: -115.30   Polyak: 0.995000   Buffer: 40.62   Loss: 121.324265  24.744286  24.608900
Ep: 221   Rew: -99.21   Avg Rew: -114.90   Polyak: 0.995000   Buffer: 40.64   Loss: 121.308212  26.982824  24.897024
Ep: 222   Rew: -100.08   Avg Rew: -114.48   Polyak: 0.9950

Ep: 284   Rew: -105.85   Avg Rew: -102.25   Polyak: 0.995000   Buffer: 41.96   Loss: 126.098587  27.198757  27.141512
Ep: 285   Rew: -102.92   Avg Rew: -102.25   Polyak: 0.995000   Buffer: 41.97   Loss: 125.529839  28.350739  24.628132
Ep: 286   Rew: -97.56   Avg Rew: -102.17   Polyak: 0.995000   Buffer: 42.00   Loss: 126.281685  25.062733  32.384502
Ep: 287   Rew: -97.72   Avg Rew: -102.13   Polyak: 0.995000   Buffer: 42.03   Loss: 125.725853  24.991394  29.677088
Ep: 288   Rew: -105.73   Avg Rew: -102.19   Polyak: 0.995000   Buffer: 42.04   Loss: 127.033180  24.602636  27.427910
Ep: 289   Rew: -100.09   Avg Rew: -102.14   Polyak: 0.995000   Buffer: 42.06   Loss: 125.833893  27.159954  22.481808
Ep: 290   Rew: -99.75   Avg Rew: -102.14   Polyak: 0.995000   Buffer: 42.09   Loss: 128.423615  49.242580  42.115501
Ep: 291   Rew: -102.00   Avg Rew: -102.15   Polyak: 0.995000   Buffer: 42.11   Loss: 125.270348  22.135513  26.019474
Ep: 292   Rew: -98.51   Avg Rew: -102.14   Polyak: 0.995000

...

Ep: 22293   Rew: -69.03   Avg Rew: -104.13   Polyak: 0.995000   Buffer: 100.00   Loss: 17.416670  0.086798  0.107327
Ep: 22294   Rew: -105.39   Avg Rew: -104.02   Polyak: 0.995000   Buffer: 100.00   Loss: 17.795542  0.177012  0.121956
Ep: 22295   Rew: -107.47   Avg Rew: -104.12   Polyak: 0.995000   Buffer: 100.00   Loss: 16.940121  0.345868  0.135636
Ep: 22296   Rew: -66.62   Avg Rew: -103.31   Polyak: 0.995000   Buffer: 100.00   Loss: 17.284203  0.198691  0.160835
Ep: 22297   Rew: -131.59   Avg Rew: -103.55   Polyak: 0.995000   Buffer: 100.00   Loss: 17.463499  0.187077  0.160052
Ep: 22298   Rew: -53.17   Avg Rew: -103.07   Polyak: 0.995000   Buffer: 100.00   Loss: 17.518259  0.110382  0.111118
Ep: 22299   Rew: -114.78   Avg Rew: -103.47   Polyak: 0.995000   Buffer: 100.00   Loss: 17.157198  0.146124  0.168765
Ep: 22300   Rew: -125.85   Avg Rew: -103.75   Polyak: 0.995000   Buffer: 100.00   Loss: 18.398270  0.220294  0.135268
Ep: 22301   Rew: -80.88   Avg Rew: -103.51   Polyak: 0.9950

Ep: 22363   Rew: -109.45   Avg Rew: -110.24   Polyak: 0.995000   Buffer: 100.00   Loss: 15.593583  0.256598  0.223343
Ep: 22364   Rew: -120.49   Avg Rew: -110.73   Polyak: 0.995000   Buffer: 100.00   Loss: 16.059610  0.360519  0.517053
Ep: 22365   Rew: -131.76   Avg Rew: -111.48   Polyak: 0.995000   Buffer: 100.00   Loss: 16.203224  1.824953  2.419069
Ep: 22366   Rew: -123.05   Avg Rew: -112.06   Polyak: 0.995000   Buffer: 100.00   Loss: 16.346348  0.880855  0.173762
Ep: 22367   Rew: -123.68   Avg Rew: -112.28   Polyak: 0.995000   Buffer: 100.00   Loss: 17.066957  0.190228  0.443494
Ep: 22368   Rew: -113.05   Avg Rew: -112.60   Polyak: 0.995000   Buffer: 100.00   Loss: 16.897982  0.353789  0.362928
Ep: 22369   Rew: -112.68   Avg Rew: -112.97   Polyak: 0.995000   Buffer: 100.00   Loss: 16.610685  0.711811  1.419264
Ep: 22370   Rew: -108.01   Avg Rew: -113.27   Polyak: 0.995000   Buffer: 100.00   Loss: 15.463792  0.083257  0.093739
Ep: 22371   Rew: -127.68   Avg Rew: -113.59   Polyak: 0.

Ep: 22433   Rew: -87.60   Avg Rew: -109.87   Polyak: 0.995000   Buffer: 100.00   Loss: 13.499596  0.062526  0.077391
Ep: 22434   Rew: -90.85   Avg Rew: -109.14   Polyak: 0.995000   Buffer: 100.00   Loss: 13.410141  0.059977  0.046743
Ep: 22435   Rew: -94.73   Avg Rew: -108.66   Polyak: 0.995000   Buffer: 100.00   Loss: 13.561503  0.181453  0.137553
Ep: 22436   Rew: -101.06   Avg Rew: -108.12   Polyak: 0.995000   Buffer: 100.00   Loss: 13.678153  0.103428  0.062480
Ep: 22437   Rew: -128.41   Avg Rew: -108.15   Polyak: 0.995000   Buffer: 100.00   Loss: 13.279230  0.089232  0.089351
Ep: 22438   Rew: -101.09   Avg Rew: -108.07   Polyak: 0.995000   Buffer: 100.00   Loss: 13.099152  0.069027  0.120997
Ep: 22439   Rew: -95.18   Avg Rew: -108.14   Polyak: 0.995000   Buffer: 100.00   Loss: 13.231863  0.062941  0.118406
Ep: 22440   Rew: -113.74   Avg Rew: -108.27   Polyak: 0.995000   Buffer: 100.00   Loss: 13.328839  0.058734  0.073336
Ep: 22441   Rew: -82.48   Avg Rew: -107.92   Polyak: 0.99500

Ep: 22504   Rew: -62.98   Avg Rew: -89.60   Polyak: 0.995000   Buffer: 100.00   Loss: 11.390296  0.024957  0.026187
Ep: 22505   Rew: -116.83   Avg Rew: -89.97   Polyak: 0.995000   Buffer: 100.00   Loss: 11.173010  0.080940  0.082413
Ep: 22506   Rew: -74.09   Avg Rew: -89.71   Polyak: 0.995000   Buffer: 100.00   Loss: 11.181491  0.023924  0.026653
Ep: 22507   Rew: -65.40   Avg Rew: -89.64   Polyak: 0.995000   Buffer: 100.00   Loss: 11.106834  0.908811  0.906633
Ep: 22508   Rew: -62.59   Avg Rew: -89.29   Polyak: 0.995000   Buffer: 100.00   Loss: 11.009366  0.025696  0.026279
Ep: 22509   Rew: -71.66   Avg Rew: -89.43   Polyak: 0.995000   Buffer: 100.00   Loss: 10.950016  0.029578  0.029825
Ep: 22510   Rew: -77.28   Avg Rew: -89.23   Polyak: 0.995000   Buffer: 100.00   Loss: 11.133527  0.036578  0.040871
Ep: 22511   Rew: -92.44   Avg Rew: -89.29   Polyak: 0.995000   Buffer: 100.00   Loss: 11.379889  0.041163  0.046020
Ep: 22512   Rew: -79.54   Avg Rew: -89.34   Polyak: 0.995000   Buffer: 

KeyboardInterrupt: 

In [ ]:
def test():  
    random_seed = 0
    n_episodes = 3
    max_timesteps = 2000
    render = True
    save_gif = True
    
    filename = "TD3_torch_{}_{}".format(env_name, random_seed)
    filename += ''
    directory = "./preTrained/td3_torch/{}".format(env_name)
    
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    
    policy = TD3(state_dim, action_dim, max_action)
    
    policy.load_actor(directory, filename)
    
    for ep in range(1, n_episodes+1):
        ep_reward = 0
        state = env.reset()
        for t in range(max_timesteps):
            action = policy.select_action(state)
            state, reward, done, _ = env.step(action)
            ep_reward += reward
            if render:
                env.render()
                if save_gif:
                    dirname = './gif/td3_torch/{}'.format(ep)
                    if not os.path.isdir(dirname):
                        os.mkdir(dirname)
                    img = env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('./gif/td3_torch/{}/{}.jpg'.format(ep,t))
            if done:
                break
            
        print('Episode: {}\tReward: {}'.format(ep, int(ep_reward)))
        ep_reward = 0
        env.close()        
                
test()
    
    